In [ ]:
%pip install scrapy 
%pip install crochet 
%pip install scrapy-selenium
%pip install pandas

In [2]:
from typing import Iterable
import scrapy
from scrapy_selenium import SeleniumRequest


class itemFormat(scrapy.Item):

    card_name = scrapy.Field()
    creature_type = scrapy.Field()
    mana_cost = scrapy.Field()
    card_text = scrapy.Field()
    legendary = scrapy.Field()
    card_type = scrapy.Field()

class databaseSpider(scrapy.Spider):
    name = "databaseSpider"

    custom_settings = {
        'FEED_FORMAT':'json',
        'FEED_URI':f'databaseSpider.json',
    }

    def start_requests(self) -> Iterable[scrapy.Request]:
        base_url = ['https://gatherer.wizards.com/Pages/Search/Default.aspx?color=|[W]|[U]|[B]|[R]|[G]']

        for url in base_url:
            yield SeleniumRequest(url = url, callback = self.parse)
        return super().start_requests()

    def parse(self, response):

        for i, card in enumerate(response.css('tr.cardItem.oddItem')):
            type_check = card.css('div.cardInfo span.typeLine::text').get(i)
            card_item = itemFormat()
            

            card_item['card_name'] = card.css('div.cardInfo span.cardTitle a::text').get(i)

            if "Creature" in type_check:

                if "\u2014" in card.css('div.cardInfo span.typeLine::text').get(i):
                    text = card.css('div.cardInfo span.typeLine::text').get(i).replace("\u2014"," ").replace("                                 "," ").replace("\r\n"," ").replace("                                 "," ").replace("Creature", " ").replace("      "," ")
                    card_item["card_type"] = "Creature"
                    card_item['creature_type'] = text

                    if "Legendary" in text:
                        card_item['legendary'] = True
                    else:
                        card_item['legendary'] = False
                        
                else:
                    card_item['card_type'] = card.css('div.cardInfo span.typeLine::text').get(i).replace("                                 "," ").replace("\r\n"," ").replace("                                 "," ")
            
            elif "Enchantment" in type_check:
                card_item['card_type'] = "Enchantment"
            elif "Aura" in type_check:
                card_item['card_type'] = "Aura"
            elif "Instant" in type_check:
                card_item['card_type'] = "Instant"
            elif "Battle" in type_check:
                card_item['card_type'] = "Battle"
            elif "Sorcery" in type_check:
                card_item['card_type'] = "Sorcery"
        

            card_item['card_text'] = card.css('div.rulesText p::text').get(i)
            card_item['mana_cost'] = card.css('span.manaCost::attr(alt)').get(i)

            yield card_item

In [ ]:
from scrapy.crawler import CrawlerProcess

process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})
    
process.crawl(databaseSpider)
process.start()

In [ ]:
import pandas

file_name = f'databaseSpider.json'
dfjson = pandas.read_json(file_name)
dfjson